Create your own notebook pages.  Use a naming convention such aas: YOURNAME-HW7-linkanalysis-partXXX with your name replacing YOURNAME.

Upload your completed jupyter notebooks to elearning site as your homework submission zip them up in directory  YOURNAME-linkanalysis-fall2017.zip . You should put these notebooks on your github as well.


7.1  Create a crawler using the webcrawler provided and discussed in my jupyter notebook ( https://github.com/pschragger/big-data-python-class/blob/master/Lectures/Lecture_7_-_Link_Analysis.ipynb) , have it crawl the first 200 pages from a base URL of your choosing

7.2  Create a Stochastic matrix from its resulting crawling as per https://cs7083.wordpress.com/2013/01/31/demystifying-the-pagerank-and-hits-algorithms/

7.3 Pass it through the Page Rank algorithm and provide the list of the top 5 page URLs in your sample

7.x For extra credit use the hits algorithm ( with a connectivity matrix )

7.xx For advanced credit find another ranking alogrithm and demonstrate that as well



7.1 Create a crawler using the webcrawler provided and discussed in my jupyter notebook ( https://github.com/pschragger/big-data-python-class/blob/master/Lectures/Lecture_7_-_Link_Analysis.ipynb) , have it crawl the first 200 pages from a base URL of your choosing

We need to create a project first.

In [ ]:
%%cmd
scrapy startproject tutorials

then directory named tutorials is created with 
tutorials/
    scrapy.cfg            # deploy configuration file

    tutorials/             # project's Python module, you'll import your code from here
        __init__.py

        items.py          # project items definition file

        pipelines.py      # project pipelines file

        settings.py       # project settings file

        spiders/          # a directory where you'll later put your spiders
            __init__.py

In [ ]:
%%cmd
scrapy genspider tutorialspoint tutorialspoint.com

In [1]:
%%writefile tutorials/tutorials/spiders/TutorialSpider.py
from scrapy.contrib.linkextractors.sgml import SgmlLinkExtractor
from scrapy.contrib.spiders import CrawlSpider, Rule
from scrapy.item import Item, Field


class TutItem(Item):
    url= Field()


class TutorialSpider(CrawlSpider):
    name = 'tutorialspoint' #identifies the name of the spider
    allowed_domains = ['tutorialspoint.com']
    start_urls = ['https://www.tutorialspoint.com/codingground.htm','https://www.tutorialspoint.com/tutorialslibrary.htm']

    rules = (Rule(SgmlLinkExtractor(), callback='parse_url', follow=False), )

    def parse_url(self, response):#gives the response for all the requests
        item = TutItem()
        item['url'] = response.url
        return item

Overwriting tutorials/tutorials/spiders/TutorialSpider.py


In [ ]:
Spiders are the classes in which we define how the webpages should be scraped.
In our TutorialSpider we are scraping the urls.

In [2]:
%%cmd
scrapy shell "https://www.tutorialspoint.com/codingground.htm"

Microsoft Windows [Version 10.0.15063]
(c) 2017 Microsoft Corporation. All rights reserved.

(C:\Users\AsusGo\Anaconda2) C:\Users\AsusGo\Desktop\big-data\Homeworks\Link_Homework>scrapy shell "https://www.tutorialspoint.com/codingground.htm"
[s] Available Scrapy objects:
[s]   scrapy     scrapy module (contains scrapy.Request, scrapy.Selector, etc)
[s]   crawler    <scrapy.crawler.Crawler object at 0x0000000005729A58>
[s]   item       {}
[s]   request    <GET https://www.tutorialspoint.com/codingground.htm>
[s]   response   <200 https://www.tutorialspoint.com/codingground.htm>
[s]   settings   <scrapy.settings.Settings object at 0x00000000057299E8>
[s]   spider     <DefaultSpider 'default' at 0x5a92cc0>
[s] Useful shortcuts:
[s]   fetch(url[, redirect=True]) Fetch URL and update local objects (by default, redirects are followed)
[s]   fetch(req)                  Fetch a scrapy.Request and update local objects 
[s]   shelp()           Shell help (print this help)
[s]   view(response)    

2017-11-19 15:07:19 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2017-11-19 15:07:19 [scrapy.utils.log] INFO: Overridden settings: {'LOGSTATS_INTERVAL': 0, 'DUPEFILTER_CLASS': 'scrapy.dupefilters.BaseDupeFilter'}
2017-11-19 15:07:19 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2017-11-19 15:07:20 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.down

In [3]:
%%cmd
cd tutorials
scrapy crawl tutorialspoint

Microsoft Windows [Version 10.0.15063]
(c) 2017 Microsoft Corporation. All rights reserved.

(C:\Users\AsusGo\Anaconda2) C:\Users\AsusGo\Desktop\big-data\Homeworks\Link_Homework>cd tutorials

(C:\Users\AsusGo\Anaconda2) C:\Users\AsusGo\Desktop\big-data\Homeworks\Link_Homework\tutorials>scrapy crawl tutorialspoint

(C:\Users\AsusGo\Anaconda2) C:\Users\AsusGo\Desktop\big-data\Homeworks\Link_Homework\tutorials>

C:\Users\AsusGo\Desktop\big-data\Homeworks\Link_Homework\tutorials\tutorials\spiders\TutorialSpider.py:1: ScrapyDeprecationWarning: Module `scrapy.contrib.linkextractors` is deprecated, use `scrapy.linkextractors` instead
  from scrapy.contrib.linkextractors.sgml import SgmlLinkExtractor
C:\Users\AsusGo\Desktop\big-data\Homeworks\Link_Homework\tutorials\tutorials\spiders\TutorialSpider.py:1: ScrapyDeprecationWarning: Module `scrapy.contrib.linkextractors.sgml` is deprecated, use `scrapy.linkextractors.sgml` instead
  from scrapy.contrib.linkextractors.sgml import SgmlLinkExtractor
C:\Users\AsusGo\Desktop\big-data\Homeworks\Link_Homework\tutorials\tutorials\spiders\TutorialSpider.py:2: ScrapyDeprecationWarning: Module `scrapy.contrib.spiders` is deprecated, use `scrapy.spiders` instead
  from scrapy.contrib.spiders import CrawlSpider, Rule
C:\Users\AsusGo\Desktop\big-data\Homeworks\Link_Homework\tutorials\tutorials\spiders\TutorialSpider.py:15: ScrapyDeprecationWarning: SgmlLinkExtracto

In [6]:
%%cmd
cd tutorials
scrapy crawl tutorialspoint -o items.csv -t csv

Microsoft Windows [Version 10.0.15063]
(c) 2017 Microsoft Corporation. All rights reserved.

(C:\Users\AsusGo\Anaconda2) C:\Users\AsusGo\Desktop\big-data\Homeworks\Link_Homework>cd tutorials

(C:\Users\AsusGo\Anaconda2) C:\Users\AsusGo\Desktop\big-data\Homeworks\Link_Homework\tutorials>scrapy crawl tutorialspoint -o items.csv -t csv

(C:\Users\AsusGo\Anaconda2) C:\Users\AsusGo\Desktop\big-data\Homeworks\Link_Homework\tutorials>

C:\Users\AsusGo\Desktop\big-data\Homeworks\Link_Homework\tutorials\tutorials\spiders\TutorialSpider.py:1: ScrapyDeprecationWarning: Module `scrapy.contrib.linkextractors` is deprecated, use `scrapy.linkextractors` instead
  from scrapy.contrib.linkextractors.sgml import SgmlLinkExtractor
C:\Users\AsusGo\Desktop\big-data\Homeworks\Link_Homework\tutorials\tutorials\spiders\TutorialSpider.py:1: ScrapyDeprecationWarning: Module `scrapy.contrib.linkextractors.sgml` is deprecated, use `scrapy.linkextractors.sgml` instead
  from scrapy.contrib.linkextractors.sgml import SgmlLinkExtractor
C:\Users\AsusGo\Desktop\big-data\Homeworks\Link_Homework\tutorials\tutorials\spiders\TutorialSpider.py:2: ScrapyDeprecationWarning: Module `scrapy.contrib.spiders` is deprecated, use `scrapy.spiders` instead
  from scrapy.contrib.spiders import CrawlSpider, Rule
C:\Users\AsusGo\Desktop\big-data\Homeworks\Link_Homework\tutorials\tutorials\spiders\TutorialSpider.py:15: ScrapyDeprecationWarning: SgmlLinkExtracto

Now we will save all url links in a items.csv file.

References:

https://doc.scrapy.org/en/latest/intro/tutorial.html

https://stackoverflow.com/questions/18903197/scrapy-xpath-all-the-links-on-the-page

https://doc.scrapy.org/en/latest/intro/tutorial.html#creating-a-project